In [1]:
import re
import sys
from pathlib import Path
from collections import Counter

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
ROOT = Path('/home/jun0/Projects/emart24')
sys.path.append(str(ROOT))
print(sys.path)

['/home/jun0/Projects/emart24/emart24_codes/RJY', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python38.zip', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8/lib-dynload', '', '/home/jun0/.local/lib/python3.8/site-packages', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda110/lib/python3.8/site-packages', '/home/jun0/.local/lib/python3.8/site-packages/IPython/extensions', '/home/jun0/.ipython', '/home/jun0/Projects/emart24']


# 분류 체계 확장

In [3]:
# 카테고리 데이터
category = pd.read_excel(ROOT / 'data/category.xlsx')
category.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']

# 상품 마스터 데이터
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})

# shapes
category.shape, item.shape

((3062, 4), (49299, 46))

In [5]:
category.head(2)

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"


In [68]:
contained = item.fillna('').apply(lambda row: (row['GDS_SLPNT_DSCRT'] in row['GDS_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
contained_reverse = item.fillna('').apply(lambda row: (row['GDS_DSCRT'] in row['GDS_SLPNT_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
only_dscrt = item['GDS_DSCRT'].isna() & ~item['GDS_SLPNT_DSCRT'].isna()
only_slpnt = ~item['GDS_DSCRT'].isna() & item['GDS_SLPNT_DSCRT'].isna()

In [88]:
item[item.fillna('').apply(lambda row: (row['GDS_SLPNT_DSCRT'] != row['GDS_DSCRT']) & (row['GDS_DSCRT'] in row['GDS_SLPNT_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
55,에너자이저)맥스AAA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
123,에너자이저)맥스AA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
704,천지개벽100ml,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...
753,하겐다즈)화이트블레스케익,"- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛...","- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛..."
9174,마즈)스니커즈땅콩헤이즐넛,"고소함으로 가득찬 헤이즐넛, 두개로 나누어 먹을 수 있는 초코바","1. 고소함으로 가득찬 헤이즐넛, 두개로 나누어 먹을 수 있는 초코바 2. 학원가..."
...,...,...,...
48592,스티커)컬러풀하트스티커,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 다양한 모양의 하트 스티...,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 다양한 모양의 하트 스티...
48593,스티커)마스크데코스티커(과일),1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 밋밋한 마스크를 개성있게...,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 밋밋한 마스크를 개성있게...
48594,바른생각)콘돔익스트림에어핏기획(1P),1.바른생각)인기콘돔4종클립스트립(기획상품) 구성품 / 상품 발주 불가,1.바른생각)인기콘돔4종클립스트립(기획상품) 구성품 / 상품 발주 불가 4. 진열...
49086,신스무디킹)글래드랩(포장용),1. 스무디킹 운영용 소모품 재료,1. 스무디킹 운영용 소모품 재료


In [84]:
item[item['GDS_DSCRT'].isna() & item['GDS_SLPNT_DSCRT'].isna()].shape

(19670, 48)

In [81]:
item[item.fillna('').apply(lambda row: bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)].shape

(13869, 48)

In [83]:
3624 / 13869

0.2613021847285313

In [69]:
item[contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(3578, 3)

In [70]:
item[only_dscrt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(8, 3)

In [71]:
item[only_slpnt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(15752, 3)

In [72]:
item[contained_reverse][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(3624, 3)

In [73]:
item[~contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(45721, 3)

In [9]:
temp = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
temp[temp['점포발주입수수량'].isna() | temp['공급가'].isna() | temp['매가'].isna() | temp['점포유통기한일수'].isna()][['점포발주입수수량', '공급가', '매가', '점포유통기한일수']]

,점포발주입수수량,공급가,매가,점포유통기한일수
27089,NaN,333,450,NaN


In [41]:
# 전체 카테고리
all_category = []
temp = pd.DataFrame(category['전체분류'].apply(lambda row: row.split('>')).tolist())
for col in temp.columns:
    all_category += temp[col].unique().tolist()
unique_category = set(all_category)
len(all_category), len(unique_category)

(3576, 3427)

In [4]:
# 전체분류 마지막 항목과 현재분류명 일치하지 않는 케이스
category[~(category['전체분류'].apply(lambda row: row.split('>')[-1]) == category['현재분류명'])]

,KAN_CODE,전체분류,현재분류명,상품예시


In [35]:
# 전처리해야할 대상들
category[category['전체분류'].duplicated(keep=False)]

,KAN_CODE,전체분류,현재분류명,상품예시


In [42]:
# category의 아이템
category_item = [x for lst in category['상품예시'].apply(lambda row: [x.strip() for x in row.split(',')]).tolist() for x in lst]
# '' 제거
category_item = list(filter(lambda x: x != '', category_item))
category_item_unique = set(category_item)
category_item_unique = list(category_item_unique)[1:]
len(category_item), len(category_item_unique)

(9353, 9140)

In [45]:
# unique 한 상품명
item_unique_bool = ~item['상품명'].duplicated()
# 분류가 겹치는 아이템
overlap = item_unique_bool & (item['대분류명'].isin(unique_category) | item['중분류명'].isin(unique_category) | item['소분류명'].isin(unique_category))
# 단순 일치하는 상품명
exact_match_bool = item_unique_bool & item['상품명'].isin(category_item_unique)
# 상품명에 포함된 상품예시
contain_bool = item_unique_bool & item['상품명'].apply(lambda x: bool([i for i in category_item_unique if i in x]))
item['상품예시'] = item['상품명'].apply(lambda x: [i for i in category_item_unique if i in x])
item['카테고리'] = ''

item[overlap].shape, item[exact_match_bool].shape, item[contain_bool].shape, item[overlap | contain_bool].shape

((17830, 47), (283, 47), (23738, 47), (31591, 47))

In [66]:
item[item['상품예시'].apply(lambda row: bool([x for x in row if len(x) > 1]))][['PLU_CD', '상품명', '상품예시', '카테고리']].sample(100)

,PLU_CD,상품명,상품예시,카테고리
36017,8801062896943,롯데)스키피땅콩스트로젤,[땅콩],
10781,8806075265885,자주)물병에코(오렌지),"[물병, 오렌지]",
16570,8809073333556,완구)호키동물피아노,[피아노],
40371,8801198180282,영실업)마이크로휠(싱글팩),"[마이크, 마, 실]",
17101,8803284608199,피코크)리치핫초코10입,[리치],
...,...,...,...,...
11565,8801047163053,명품)한우야채죽,"[야채죽, 한우]",
29534,8801047154761,동원)돌후룻샐러드파우치115g,[샐러드],
42186,5702016618259,레고)21158 판다 가족의 놀이터,[레고],
32226,1500000012847,바르다)참치&크림치즈호두김밥,"[호두, 크림치즈]",


In [92]:
temp_item_list = ['징', '오일', '클렌징오일', '참기름', '수세미', '시가', '주방가위', '가위']
temp_bool = category['상품예시'].apply(lambda row: any([x.strip() in temp_item_list for x in row.split(',')]))
category[temp_bool].head(10)

,KAN_CODE,전체분류,현재분류명,상품예시
36,1010501,가공식품>조미료>유지류>참기름,참기름,"참기름, 생참기름, 맛참기름"
755,3110106,일상용품>주방용품>조리기구>주방가위,주방가위,"주방가위, 식가위, 식사가위"
793,3110405,일상용품>주방용품>주방소모품>수세미,수세미,"수세미, 철수세미, 천수세미"
943,3180499,일상용품>공구>절삭공구>기타절삭공구,기타절삭공구,"가위, 트리머, 다이스, 대패, 루터, 샌더기, 톱, 조각기, 직소기, 커터, 탭"
1040,3200305,일상용품>화장품>스킨케어화장품>오일,오일,"오일, 스킨오일, 페이셜오일"
1178,3201407,일상용품>화장품>클렌징>클렌징오일,클렌징오일,"클렌징오일, 오일형클렌져, 클렌져오일"
1197,3220103,일상용품>담배용품>담배>시가,시가,"시가, 바닐라시가, 플레이버시가"
1495,6010207,교육/문화용품>문구/사무용품>일반사무용품>가위,가위,"가위, 문서가위, 사무용가위"
1769,6040499,교육/문화용품>악기>국악악기>기타국악악기,기타국악악기,"대취타, 소고, 징"
1783,6040704,교육/문화용품>악기>타악기>징,징,"징, 국악용징, 학생용징"


In [ ]:
# 상품 설명 보기
item[['GDS_DSCRT', 'GDS_SLPNT_DSCRT']].drop_duplicates().values.tolist()

# 분류 체계 매핑

In [4]:
# 거래데이터
# emart24 = ROOT / 'data/emart24Data'
# f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')
transaction = pd.DataFrame()
for i, file in enumerate(files):
    transaction = pd.concat([transaction, pd.read_csv(file, sep='|', low_memory=False)], 0)
    break

# 파일 수, 데이터 건수
i + 1, transaction.shape

/tmp/ipykernel_2359845/2573596981.py:11: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  transaction = pd.concat([transaction, pd.read_csv(file, sep='|', low_memory=False)], 0)


(1, (361088, 95))

In [9]:
transaction.iloc[:, :13]

,TRAN_YMD,COM_CD,STORE_CD,POS_NO,TRAN_NO,SYS_YMDHMS,TRAN_TYPE,TRAN_TYPE_NM,TRAN_KIND,TRAN_KIND_NM,PMOD_YN,STOP_YN,RFND_YN
0,20190101,1002,4890.0,1,1665,20190101000303,0,통상,0,판매,0,0,NaN
1,20190101,1002,4890.0,1,1666,20190101000352,0,통상,0,판매,0,0,NaN
2,20190101,1002,4890.0,1,1668,20190101000810,0,통상,0,판매,0,0,NaN
3,20190101,1002,4890.0,1,1670,20190101001001,0,통상,0,판매,0,0,NaN
4,20190101,1002,4890.0,1,1670,20190101001001,0,통상,0,판매,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361083,20210131,1002,4890.0,1,2787,20210131231213,0,통상,0,판매,0,0,NaN
361084,20210131,1002,4890.0,1,2789,20210131233145,0,통상,0,판매,0,0,NaN
361085,20210131,1002,4890.0,1,2790,20210131233725,0,통상,0,판매,0,0,NaN
361086,20210131,1002,4890.0,1,2790,20210131233725,0,통상,0,판매,0,0,NaN


In [10]:
transaction.iloc[:, 13:26]

,OP_NO,ORG_TRAN_YMD,ORG_STORE_CD,ORG_POS_NO,ORG_TRAN_NO,ORG_RES_ID,APP_OP_NO,CUST_CLS_ID,CUST_COUNT,ORDER_TYPE,FLOOR_CD,TABLE_NO,GSALE_AMT
0,4890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,4700.0
1,4890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,4500.0
2,4890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,9000.0
3,4890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,32400.0
4,4890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,32400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361083,4907,NaN,NaN,NaN,NaN,NaN,4907.0,NaN,1,NaN,NaN,NaN,5000.0
361084,4907,NaN,NaN,NaN,NaN,NaN,4907.0,NaN,1,NaN,NaN,NaN,5200.0
361085,4907,NaN,NaN,NaN,NaN,NaN,4907.0,NaN,1,NaN,NaN,NaN,9900.0
361086,4907,NaN,NaN,NaN,NaN,NaN,4907.0,NaN,1,NaN,NaN,NaN,9900.0


In [11]:
transaction.iloc[:, 26:39]

,GDC_AMT,NSALE_AMT,SCHARGE_AMT,DRAWER_CD,RECT_ISS_YMDHMS,REG_YMDHMS,PROC_ID,PROC_YMDHMS,PROC_CNT,PROC_PNO,LOAD_DTM,SYS_YMD,ITEM_SEQ
0,0.0,4700,0,NaN,20190101000303,20190101000313,1.0,2019010100040950337,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1
1,0.0,4500,0,NaN,20190101000352,20190101000358,1.0,2019010100040950337,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1
2,0.0,9000,0,NaN,20190101000810,20190101000817,1.0,2019010100090997399,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1
3,5400.0,27000,0,NaN,20190101001001,20190101001008,1.0,2019010100241833223,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,1
4,5400.0,27000,0,NaN,20190101001001,20190101001008,1.0,2019010100241833223,NaN,NaN,03-JAN-19 03.02.09.000000 AM,20181231,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361083,2800.0,5000,0,NaN,20210131231213,20210131231232,0.0,2021013123123200000,NaN,NaN,05-FEB-21 03.01.41.000000 PM,20210131,4
361084,1900.0,5200,0,NaN,20210131233145,20210131233211,0.0,2021013123321100000,NaN,NaN,05-FEB-21 03.01.41.000000 PM,20210131,1
361085,0.0,9900,0,NaN,20210131233725,20210131233911,0.0,2021013123391100000,NaN,NaN,05-FEB-21 03.01.41.000000 PM,20210131,1
361086,0.0,9900,0,NaN,20210131233725,20210131233911,0.0,2021013123391100000,NaN,NaN,05-FEB-21 03.01.41.000000 PM,20210131,3


In [12]:
transaction.iloc[:, 39:52]

,ASGN_CANCEL_YN,ITEM_CD,PLU_CD,PLU_FLAG,CATE_ID,GRP_ID,SGRP_ID,BRAND_ID,ITEM_ID,ITEM_NM,SALE_QTY,SALE_UPRC,SALE_AMT
0,0.0,D118801094202606,8.801094e+12,0.0,107.0,117.0,1002.0,A1300001,0.0,스프라이트500ml,1.0,1800.0,1800.0
1,0.0,D110000088017624,8.801762e+07,0.0,113.0,151.0,1003.0,A1300001,0.0,P)팔리아멘트하이브리드5mg,1.0,4500.0,4500.0
2,0.0,D118801116001712,8.801116e+12,0.0,113.0,143.0,1001.0,A1300001,0.0,K)에쎄수0.1,2.0,4500.0,9000.0
3,0.0,D112201035603232,6.901036e+12,0.0,110.0,133.0,1004.0,A1300001,0.0,비어케이)칭따오캔맥주500ML,1.0,3200.0,3200.0
4,0.0,D112201004006813,4.901004e+12,0.0,110.0,133.0,1004.0,A1300001,0.0,롯데아사히)캔맥주500ML,1.0,3900.0,3900.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361083,0.0,D118809600251780,8.809600e+12,0.0,201.0,203.0,2001.0,A1300001,0.0,스팸김치볶음삼각,1.0,1100.0,1100.0
361084,0.0,D118801094202408,8.801094e+12,0.0,208.0,206.0,2006.0,A1300001,0.0,스프라이트1.5L,1.0,2900.0,2900.0
361085,0.0,D112301858011017,8.801858e+12,0.0,110.0,133.0,1001.0,A1300001,0.0,카스후레쉬캔맥주355ML,2.0,1650.0,3300.0
361086,0.0,D118801073310704,8.801073e+12,0.0,209.0,208.0,2004.0,A1300001,0.0,삼양)별뽀빠이라면스낵,2.0,800.0,1600.0
